In [1]:
import os
import ast
from tqdm import tqdm
import json
from glob import glob
from collections import Counter
import sys
sys.path.insert(0, '../')

import pandas as pd
import numpy as np

from load_data import load
from tfidf import load_tfidf
from config import Config
from utils import iterate_data_files

In [2]:
tfidf_dir = Config.tfidf_root
vocab_dir = Config.vocab_root
data_root = Config.data_root

In [3]:
load_tfidf(post_meta_id_list=[1,2,3], tfidf_dir=tfidf_dir, vocab_dir=vocab_dir, drop_id=False)

,post_meta_id,여행,영화,사랑,에세이,그림일기,생각,공감에세이,일상,글쓰기,감성에세이,인생,행복,연애,소설,드로잉,독서,육아,그림,마음,사진,리뷰,엄마,음악,스타트업,시,유럽여행,일러스트,이별,마케팅,고양이,디자인,영화리뷰,웹툰,IT,직장인,감성,일기,사람,결혼,...,식도락,서민,F1,라디오헤드,연민,운동방법,여교사,하체,한약,족저근막염,좋은음악,총기,폭탄,도파민,산행후기,간병인,시화,고요함,멤버십,푸조,유럽연합,매체,눈높이,댄서,대상포진,블랙박스,불가능,연수,외침,신트라,삿포로여행,김수현,고부갈등,배달음식,의류,본인,체험학습,선물투자,시금치,벌금
0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.268116,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
START_DEV = '2018100100'
END_DEV = '2018101000'
ROOT = '../raw'

In [7]:
post_meta = pd.read_csv(os.path.join(ROOT, '../raw/preprocessed/post_meta_id.csv'))

post_meta_id = post_meta['post_meta_id'].tolist()
post_id = post_meta['post_id'].tolist()

post_id_encoding = {i:j for i, j in zip(post_id, post_meta_id)}
post_id_decoding = {i:j for i, j in zip(post_meta_id, post_id)}

In [9]:
import pickle

In [12]:
with open('../raw/preprocessed/post_id_encoding.pickle', 'wb') as handle:
    pickle.dump(post_id_encoding, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [13]:
with open('../raw/preprocessed/post_id_encoding.pickle', 'rb') as handle:
    b = pickle.load(handle)

In [11]:
with open('../raw/preprocessed/post_id_decoding.pickle', 'wb') as handle:
    pickle.dump(post_id_decoding, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [8]:
post_id_encoding

shot_516': 128,
 '@miyoungbaek_27': 129,
 '@sunmyung_45': 130,
 '@byulkim_109': 131,
 '@ljg523_1527': 132,
 '@mijeongpark_33': 133,
 '@ljg523_1520': 134,
 '@chkwon91_10': 135,
 '@mh2u_44': 136,
 '@ljg523_767': 137,
 '@mignoncat_43': 138,
 '@zzoos_20': 139,
 '@skyzakard_74': 140,
 '@elang8151_223': 141,
 '@freeoos_195': 142,
 '@pencil-k_473': 143,
 '@brunchfkx_8': 144,
 '@chaoshaha_10': 145,
 '@hintdot_47': 146,
 '@capotea_115': 147,
 '@elang8151_222': 148,
 '@youthhd_60': 149,
 '@pagan98_12': 150,
 '@chni_23': 151,
 '@m-life_284': 152,
 '@pencil-k_470': 153,
 '@uchonsuyeon_90': 154,
 '@howoonk_65': 155,
 '@brunchejej_20': 156,
 '@wikitree_1865': 157,
 '@mirejiki_30': 158,
 '@purnbd_5': 159,
 '@skyzakard_70': 160,
 '@hotelscomkr_43': 161,
 '@paak0080tz1s_5': 162,
 '@rkdls0222_11': 163,
 '@inner-beauty_264': 164,
 '@paak0080tz1s_8': 165,
 '@glassdice_25': 166,
 '@ljg523_764': 167,
 '@greatmother_1': 168,
 '@darkarkorn8cnl_423': 169,
 '@gaheepark_6': 170,
 '@gaheepark_7': 171,
 '@wikitree

In [5]:
records = [[line.strip().split() for line in open(path)] for path, _ in tqdm(iterate_data_files(from_dtm=START_DEV, to_dtm=END_DEV, root_dir=ROOT))]r

216it [00:44,  4.81it/s]


In [52]:
post_id[0]

'@bookdb_782'

In [100]:
from collections import defaultdict
user_logs = defaultdict(list) # {유저식별자 : [조회글 목록]}

In [104]:
for record in tqdm(records[:5]):
    for r in record:
        user, posts_raw = r[0], r[1:]
        posts = [post_id_dict[p] for p in posts_raw if p in post_id]
        user_logs[user].extend(posts)




  0%|          | 0/5 [00:00<?, ?it/s]


 20%|██        | 1/5 [00:28<01:55, 28.85s/it]


 40%|████      | 2/5 [01:28<01:53, 37.98s/it]


 60%|██████    | 3/5 [01:53<01:08, 34.28s/it]


 80%|████████  | 4/5 [02:38<00:37, 37.55s/it]


100%|██████████| 5/5 [03:19<00:00, 39.94s/it]


In [111]:
sample_id_list = user_logs['#fb3b42b92d96f00afd5bf62d3575db53']

In [112]:
tfidf = load_tfidf(post_meta_id_list=sample_id_list, tfidf_dir=tfidf_dir, vocab_dir=vocab_dir, drop_id=True)

In [113]:
tfidf = tfidf.apply(lambda x: x / np.linalg.norm(x), axis=1)

In [114]:
user_preference = tfidf.sum(axis=0)

In [121]:
user_preference = user_preference / np.linalg.norm(user_preference)

In [122]:
user_preference

여행      0.000000
영화      0.000000
사랑      0.186879
에세이     0.000000
그림일기    0.217054
          ...   
본인      0.000000
체험학습    0.000000
선물투자    0.000000
시금치     0.000000
벌금      0.000000
Length: 7000, dtype: float64

In [135]:
post_subsample = np.random.choice(600000, 1000, replace=False).tolist() # 가중 확률도 괜찮겠다

In [137]:
tfidf_subsample = load_tfidf(post_meta_id_list=post_subsample, tfidf_dir=tfidf_dir, vocab_dir=vocab_dir, drop_id=True)

array([0.        , 0.        , 0.18687906, ..., 0.        , 0.        ,
       0.        ])

In [151]:
recommend_list = (tfidf_subsample * user_preference.values).sum(axis=1).sort_values(ascending=False).head(100).index.tolist()

In [154]:
recommend_list_decoded = [post_id_decode[i] for i in recommend_list]

In [155]:
recommend_list_decoded

['@tallguy_213',
 '@sharppen01_20',
 '@moonwindtre_240',
 '@bookfit_3642',
 '@haninorway19_103',
 '@ameliekim2016_38',
 '@chkwon91_10',
 '@mobiinside_559',
 '@leedongdong_58',
 '@psychorops_48',
 '@slist_86',
 '@drawingbutler_10',
 '@iampact_27',
 '@begini_379',
 '@ranrandambee_53',
 '@triple_26',
 '@littlechamber_16',
 '@ddaddassi_112',
 '@peregrino97_872',
 '@ubermensch_29',
 '@theedit_366',
 '@capotea_118',
 '@tomssi_55',
 '@antifreezie_2',
 '@yooym2000_162',
 '@byulkim_102',
 '@mori3_52',
 '@tiara8743_31',
 '@kateyoungheekwo_26',
 '@byulkim_106',
 '@eunmikim_146',
 '@jamon_2',
 '@hotelscomkr_45',
 '@9320km_50',
 '@csco2pw2gt_27',
 '@simplestories_63',
 '@bonfire_331',
 '@gunbbang_408',
 '@sound-travel_42',
 '@elang8151_222',
 '@simplestories_65',
 '@kohwang56_87',
 '@vhfhflya_45',
 '@glassdice_22',
 '@futurewave_492',
 '@garbar_384',
 '@lasw2324_28',
 '@eewol_41',
 '@gipyung_159',
 '@doyeonsunim_168',
 '@brunchfkx_8',
 '@marupress_149',
 '@byulkim_105',
 '@rana-na_26',
 '@kimssiban